In [1]:
!pip install streamlit pyngrok joblib nltk scikit-learn dill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.2 MB/s eta 0:00:00


In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("#") # Set authorization token from ngrok


In [ ]:
%%writefile app.py
import streamlit as st
import joblib
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Download stopwords
nltk.download("stopwords")

def whitespace_split(text):
    return text.split()

# Load the saved model objects (vectorizer, tfidf_transformer, stacking_model)
model_path = "/content/stack_2_models_optimized.pkl"  # Ensure this file is in your Colab working directory

vectorizer, tfidf_transformer, stacking_model = joblib.load(model_path)

def predict_review_probability(review, fake_threshold=0.7):
    review_counts = vectorizer.transform([review])
    review_tfidf = tfidf_transformer.transform(review_counts)
    probas = stacking_model.predict_proba(review_tfidf)[0]

    new_real_prob = probas[1]
    new_fake_prob = probas[0]

    prediction = "Fake Review" if new_fake_prob >= fake_threshold else "Real Review"
    return prediction, (new_real_prob, new_fake_prob)

st.title("Fake Review Detector")
st.markdown("Enter a product review below to check whether it is **Real** or **Fake**.")

review_input = st.text_area("Review Text", height=150)

if st.button("Analyze Review"):
    if not review_input.strip():
        st.error("Please enter a review.")
    else:
        prediction, (real_prob, fake_prob) = predict_review_probability(review_input, fake_threshold=0.7)
        st.markdown(f"**Probability of being Real:** {real_prob:.2%}")
        st.markdown(f"**Probability of being Fake:** {fake_prob:.2%}")
        st.markdown(f"**Prediction:** {prediction}")
        st.progress(int(fake_prob * 100))


Writing app.py


In [ ]:
from pyngrok import ngrok
import subprocess
import time

# Kill any pre-existing ngrok processes
!pkill ngrok

# Set the Streamlit port (default is 8501)
port = 8501

# Open an ngrok tunnel to the streamlit port
public_url = ngrok.connect(port).public_url
print("Your Streamlit app is available at:", public_url)

# Run the Streamlit app in a background process
# Use subprocess.Popen so that the cell does not block execution.
process = subprocess.Popen(["streamlit", "run", "app.py"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Optional: Wait a few seconds to let the app start properly
time.sleep(5)
